In [1]:
!pip install opencv-python-headless
!pip install matplotlib
!pip install Pillow
!pip install tabulate


# Import

In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!ls /content/drive/MyDrive/ODL_DB
!unzip -o /content/drive/MyDrive/ODL_DB/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET.zip -d /content/dataset

# Directory where images are extracted
image_dir = '/content/dataset'

# List all files in the directory
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('jpeg', 'jpg', 'png', 'gif'))]

# # Load images
images = []
for image_file in image_files:
    img = Image.open(image_file)
    images.append(img)

train_data_dir = '/content/dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train'
test_data_dir = '/content/dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/test'


Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train/non fire/Non_Fire (6154).jpg  
  inflating: /content/dataset/__MACOSX/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train/non fire/._Non_Fire (6154).jpg  
  inflating: /content/dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train/non fire/Non_Fire (37).png  
  inflating: /content/dataset/__MACOSX/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train/non fire/._Non_Fire (37).png  
  inflating: /content/dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train/non fire/Non_Fire (8849).jpg  
  inflating: /content/dataset/__MACOSX/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train/non fire/._Non_Fire (8849).jpg  
  inflating: /content/dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train/non fire/Non_Fire (4493).jpg  
  inflating: /content/dataset/__MACOSX/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train/non fire/._Non_Fire (4493).jpg  
  inflating: /content/dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
#PATH PROCESS
import os.path
from pathlib import Path
import glob
import seaborn as sns
import cv2

#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.feature import local_binary_pattern
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN, LSTM, GlobalAveragePooling2D, SeparableConv2D
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model

#SKLEARN CLASSIFIER
# from xgboost import XGBClassifier, XGBRegressor
# from lightgbm import LGBMClassifier, LGBMRegressor
# from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from tensorflow.keras.callbacks import LearningRateScheduler
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning)
filterwarnings("ignore", category=UserWarning)

from tabulate import tabulate  # Import tabulate for table formatting
from skimage.feature import local_binary_pattern


# Class Distribution

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)


batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)


ori_datagen  = ImageDataGenerator(rescale=1./255)
ori_train = ori_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

ori_test = ori_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Class distribution of original image

In [ ]:
# Check Class Distribution
def plot_class_distribution(generator, title):
    class_labels = list(generator.class_indices.keys())
    class_counts = np.bincount(generator.classes)

    plt.figure(figsize=(10, 5))
    bars = plt.bar(class_labels, class_counts)
    plt.xlabel('Class')
    plt.ylabel('Number of Images')
    plt.title(title)

    # Add counts on top of the bars
    for bar, count in zip(bars, class_counts):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height, str(count), ha='center', va='bottom')

    plt.show()

# Image Statistics
def calculate_image_statistics(generator, num_batches=10):
    all_images = []
    for _ in range(num_batches):
        images, _ = next(generator)
        all_images.append(images)

    all_images = np.concatenate(all_images)
    mean = np.mean(all_images, axis=(0, 1, 2))
    std = np.std(all_images, axis=(0, 1, 2))

    print(f'Mean Pixel Value: {mean}')
    print(f'Standard Deviation of Pixel Values: {std}')

# Plot Class Distribution for Train and Test Generators
plot_class_distribution(ori_train, 'Class Distribution in Training Set')
plot_class_distribution(ori_test, 'Class Distribution in Test Set')
# Calculate and Print Image Statistics for Training and Test Generators
print("Training Set Image Statistics:")
calculate_image_statistics(train_generator)

print("\nTest Set Image Statistics:")
calculate_image_statistics(test_generator)


Class distribution of Augmented image

In [ ]:
# Check Class Distribution
def plot_class_distribution(generator, title):
    class_labels = list(generator.class_indices.keys())
    class_counts = np.bincount(generator.classes)

    plt.figure(figsize=(10, 5))
    bars = plt.bar(class_labels, class_counts)
    plt.xlabel('Class')
    plt.ylabel('Number of Images')
    plt.title(title)

    # Add counts on top of the bars
    for bar, count in zip(bars, class_counts):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height, str(count), ha='center', va='bottom')

    plt.show()

# Image Statistics
def calculate_image_statistics(generator, num_batches=10):
    all_images = []
    for _ in range(num_batches):
        images, _ = next(generator)
        all_images.append(images)

    all_images = np.concatenate(all_images)
    mean = np.mean(all_images, axis=(0, 1, 2))
    std = np.std(all_images, axis=(0, 1, 2))

    print(f'Mean Pixel Value: {mean}')
    print(f'Standard Deviation of Pixel Values: {std}')

# Plot Class Distribution for Train and Test Generators

plot_class_distribution(train_generator, 'Class Distribution in Training Set')
plot_class_distribution(test_generator, 'Class Distribution in Test Set')
# Calculate and Print Image Statistics for Training and Test Generators
print("Training Set Image Statistics:")
calculate_image_statistics(train_generator)

print("\nTest Set Image Statistics:")
calculate_image_statistics(test_generator)


# Image Dimension

In [ ]:
# Recursively list all image files in the directory and subdirectories, excluding macOS system files
image_files = []
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.lower().endswith(('jpeg', 'jpg', 'png', 'gif')) and not file.startswith('._'):
            image_files.append(os.path.join(root, file))

# Initialize a list to store image dimensions
image_dimensions = []

# Load images and collect dimensions
for image_file in image_files:
    try:
        with Image.open(image_file) as img:
            width, height = img.size
            channels = len(img.getbands())  # Number of color channels
            image_dimensions.append((image_file, width, height, channels))
    except Exception as e:
        print(f"Error loading image {image_file}: {e}")


# Convert the list to a DataFrame for easy analysis
df_image_dimensions = pd.DataFrame(image_dimensions, columns=['File', 'Width', 'Height', 'Channels'])

# Display the first few rows of the DataFrame
print("First few rows of image dimensions:")
print(df_image_dimensions.head().to_string(index=False))

# Summary statistics of the image dimensions
summary_stats = df_image_dimensions.describe()

# Check for the most common dimensions and channels
common_dimensions = df_image_dimensions.groupby(['Width', 'Height', 'Channels']).size().reset_index(name='Count')
common_dimensions = common_dimensions.sort_values(by='Count', ascending=False).head(10)  # Show top 10 most common

# Display the summary statistics and the most common image dimensions in tables
print("\nSummary statistics of image dimensions:")
print(tabulate(summary_stats, headers='keys', tablefmt='psql'))  # Print summary stats using tabulate

print("\nMost common image dimensions:")
print(tabulate(common_dimensions, headers='keys', tablefmt='psql'))

# Visualize the distribution of image dimensions
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_image_dimensions, x='Width', y='Height', hue='Channels')
plt.title('Image Dimensions')
plt.xlabel('Width')
plt.ylabel('Height')
plt.show()


In [ ]:
def display_images_from_each_class(generator, num_samples_per_class=5):
    class_labels = list(generator.class_indices.keys())
    num_classes = len(class_labels)

    # Initialize dictionary to store images for each class
    images_per_class = {label: [] for label in class_labels}

    # Loop until we have enough images for each class
    while any(len(images_per_class[label]) < num_samples_per_class for label in class_labels):
        images, labels = next(generator)
        for img, label in zip(images, labels):
            class_index = np.argmax(label)
            class_name = class_labels[class_index]
            if len(images_per_class[class_name]) < num_samples_per_class:
                images_per_class[class_name].append(img)

    for class_name in class_labels:
        plt.figure(figsize=(15, 5))
        plt.suptitle(class_name, fontsize=20, fontweight='bold')
        for j in range(num_samples_per_class):
            plt.subplot(1, num_samples_per_class, j + 1)
            plt.imshow(images_per_class[class_name][j])
            plt.axis('off')
        plt.tight_layout(rect=[0, 0, 1, 0.9])  # Adjust layout to make space for the title
        plt.show()

display_images_from_each_class(ori_train, num_samples_per_class=5)
display_images_from_each_class(train_generator, num_samples_per_class=5)

In [ ]:
def display_images_from_each_class(generator, num_samples_per_class=5):
    class_labels = list(generator.class_indices.keys())
    num_classes = len(class_labels)

    # Initialize dictionary to store images for each class
    images_per_class = {label: [] for label in class_labels}

    # Loop until we have enough images for each class
    while any(len(images_per_class[label]) < num_samples_per_class for label in class_labels):
        images, labels = next(generator)
        for img, label in zip(images, labels):
            class_index = np.argmax(label)
            class_name = class_labels[class_index]
            if len(images_per_class[class_name]) < num_samples_per_class:
                images_per_class[class_name].append(img)

    for class_name in class_labels:
        plt.figure(figsize=(15, 5))
        plt.suptitle(class_name, fontsize=20, fontweight='bold')
        for j in range(num_samples_per_class):
            plt.subplot(1, num_samples_per_class, j + 1)
            plt.imshow(images_per_class[class_name][j])
            plt.axis('off')
        plt.tight_layout(rect=[0, 0, 1, 0.9])  # Adjust layout to make space for the title
        plt.show()

# Assuming train_generator is defined
display_images_from_each_class(train_generator, num_samples_per_class=5)

In [ ]:
# Function to identify edge cases based on image dimensions
def identify_edge_cases(df_image_dimensions):
    # Filter images with unusual dimensions (e.g., very small or very large)
    unusual_dimensions = df_image_dimensions[(df_image_dimensions['Width'] < 200) |
                                             (df_image_dimensions['Height'] < 200) |
                                             (df_image_dimensions['Width'] > 2000) |
                                             (df_image_dimensions['Height'] > 2000)]
    return unusual_dimensions

# Example code to load image dimensions from a directory
image_dimensions = []
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.lower().endswith(('jpeg', 'jpg', 'png', 'gif')) and not file.startswith('._'):
            image_file = os.path.join(root, file)
            try:
                with Image.open(image_file) as img:
                    width, height = img.size
                    channels = len(img.getbands())  # Number of color channels
                    image_dimensions.append({'File': image_file, 'Width': width, 'Height': height, 'Channels': channels})
            except Exception as e:
                print(f"Error loading image {image_file}: {e}")

# Convert the list to a DataFrame for easy analysis
df_image_dimensions = pd.DataFrame(image_dimensions)

# Identify edge cases
edge_cases = identify_edge_cases(df_image_dimensions)

# Display edge cases with improved debugging
def display_edge_cases(edge_cases, n=5):
    if edge_cases.empty:
        print("No edge cases found.")
        return

    # Calculate the number of rows needed for the subplots
    rows = (n + 4) // 5  # Round up division

    plt.figure(figsize=(15, rows * 5))

    for i, (_, row) in enumerate(edge_cases.iterrows()):
        if i >= n:
            break
        try:
            img = Image.open(row['File'])
            plt.subplot(rows, 5, i + 1)
            plt.imshow(img)
            plt.title(f"{row['Width']}x{row['Height']} - {os.path.basename(os.path.dirname(row['File']))}")
            plt.axis('off')
        except Exception as e:
            print(f"Error displaying image {row['File']}: {e}")

    plt.tight_layout()
    plt.show()

# Display a few edge cases
display_edge_cases(edge_cases)

In [ ]:
# Function to load images and their corresponding class labels
def load_images_and_labels(data_dir):
    image_files = []
    labels = []
    class_labels = []

    # Get class labels based on directory names
    dir_contents = os.listdir(data_dir)
    for item in dir_contents:
        if os.path.isdir(os.path.join(data_dir, item)):
            class_labels.append(item)

    for class_label in class_labels:
        class_dir = os.path.join(data_dir, class_label)
        if os.path.isdir(class_dir):
            added_label = False  # Flag to track if class_label has been added
            for file in os.listdir(class_dir):
                if file.endswith(('jpeg', 'jpg', 'png', 'gif')):
                    image_files.append(os.path.join(class_dir, file))
                    if not added_label:
                        labels.append(class_label)  # Assign class_label only once per directory
                        added_label = True


    return image_files, labels


def apply_lbp_analysis(image_files, class_labels, n=5):
    plt.figure(figsize=(15, 10))

    for i in range(3):
        img = cv2.imread(image_files[i], cv2.IMREAD_GRAYSCALE)

        # Compute Canny edges
        edges = cv2.Canny(img, 100, 200)

        # Compute LBP
        radius = 1
        n_points = 8 * radius
        lbp = local_binary_pattern(img, n_points, radius, method='uniform')

        # Plot original image
        plt.subplot(3, n, i + 1)
        plt.imshow(img, cmap='gray')
        plt.title(f'Class: {class_labels[i]}\nOriginal Image')
        plt.axis('off')

        # Plot Canny edges
        plt.subplot(3, n, i + 1 + n)
        plt.imshow(edges, cmap='gray')
        plt.title('Canny Edges')
        plt.axis('off')

        # Plot LBP image
        plt.subplot(3, n, i + 1 + 2 * n)
        plt.imshow(lbp, cmap='gray')
        plt.title('LBP Texture')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

train_image_files, train_class_labels = load_images_and_labels(train_data_dir)

# Apply Canny edge detection and LBP analysis to a few images
apply_lbp_analysis(train_image_files, train_class_labels, n=5)

In [ ]:
# Function to load images and their labels
def load_images_and_labels(data_dir):
    image_files = []
    labels = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(('jpeg', 'jpg', 'png', 'gif')):
                image_files.append(os.path.join(root, file))
                labels.append(root.split('/')[-1])  # Assuming the label is the folder name
    return image_files, labels

# Load training images and labels
train_image_files, train_labels = load_images_and_labels(train_data_dir)

# Function to calculate channel average
def calculate_channel_average(img, channel):
    channel_dict = {"R": 0, "G": 1, "B": 2}
    channel_idx = channel_dict[channel]
    if img.ndim == 3 and img.shape[2] > channel_idx:
        channel_intensities = img[:, :, channel_idx].flatten()
        return np.mean(channel_intensities)
    else:
        return 0  # Return 0 for grayscale or unsupported images

# Load images and calculate channel averages
data = []
for image_file, label in zip(train_image_files, train_labels):
    img = Image.open(image_file)
    img_np = np.array(img)
    red_avg = calculate_channel_average(img_np, "R")
    green_avg = calculate_channel_average(img_np, "G")
    blue_avg = calculate_channel_average(img_np, "B")
    data.append({'Label': label, 'Red': red_avg, 'Green': green_avg, 'Blue': blue_avg})

# Create DataFrame
df = pd.DataFrame(data)

# Group by label and calculate mean intensities
mean_intensities = df.groupby("Label").mean().reset_index()

# Plotting function
def plot_channel_intensity_barplot(df, channel):
    title_dict = {"Red": "red", "Green": "green", "Blue": "blue"}
    palet_dict = {"Red": "Reds_d", "Green": "Greens_d", "Blue": "Blues_d"}
    plt.figure(figsize=(10,3))

    values = df[channel].values
    pal = sns.color_palette(palet_dict[channel], len(values))
    rank = values.argsort().argsort()
    ax = sns.barplot(x=df["Label"], y=values, palette=np.array(pal[::-1])[rank])

    plt.ylabel("Intensity")
    plt.title(f"Average {title_dict[channel]} channel intensity for classes")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=55)
    plt.show()

# Plot the channel intensities
plot_channel_intensity_barplot(mean_intensities, "Red")
plot_channel_intensity_barplot(mean_intensities, "Green")
plot_channel_intensity_barplot(mean_intensities, "Blue")

In [ ]:
# Function to load and apply histogram equalization using OpenCV
def load_and_equalize_images_cv2(directory, batch_size=5):
    image_files = [os.path.join(root, file)
                   for root, _, files in os.walk(directory)
                   for file in files if file.endswith(('jpeg', 'jpg', 'png', 'gif'))]

    for i in range(0, len(image_files), batch_size):
        batch_files = image_files[i:i + batch_size]
        equalized_images = []
        for image_file in batch_files:
            img = cv2.imread(image_file)
            if img is not None:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
                img_yuv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YUV)
                img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
                equalized_img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
                equalized_images.append(equalized_img)
        yield equalized_images

# Function to display a batch of images
def display_images(images, title):
    plt.figure(figsize=(20, 5))  # Adjust the figure size to be more compact
    for i, img in enumerate(images):
        plt.subplot(1, len(images), i + 1)  # Use dynamic layout based on number of images
        plt.imshow(img)
        plt.axis('off')
    plt.suptitle(title, fontsize=20, fontweight='bold', y=1.05)  # Increase title size and make it bold
    plt.tight_layout()  # Adjust layout
    plt.show()

# Display original images in batches
def display_original_images(data_dir, title, batch_size=5):
    image_files = [os.path.join(root, file)
                   for root, _, files in os.walk(data_dir)
                   for file in files if file.endswith(('jpeg', 'jpg', 'png', 'gif'))]

    for i in range(0, len(image_files), batch_size):
        batch_files = image_files[i:i + batch_size]
        images = []
        for image_file in batch_files:
            img = Image.open(image_file).convert('RGB')
            images.append(img)
        display_images(images, title)
        break  # Display only the first batch

# Display equalized images in batches
def process_and_display_images(data_dir, title, batch_size=5):
    for equalized_images in load_and_equalize_images_cv2(data_dir, batch_size=batch_size):
        display_images(equalized_images, title)
        break  # Display only the first batch

# Display original train images
display_original_images(train_data_dir, 'Original Train Images')

# Display equalized train images
process_and_display_images(train_data_dir, 'Equalized Train Images')


# Intensity After Equalized Train

In [ ]:
# Function to load images and their labels
def load_images_and_labels(data_dir):
    image_files = []
    labels = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(('jpeg', 'jpg', 'png', 'gif')):
                image_files.append(os.path.join(root, file))
                labels.append(root.split('/')[-1])  # Assuming the label is the folder name
    return image_files, labels

# Function to load and apply histogram equalization using OpenCV
def equalize_image_cv2(image_file):
    img = cv2.imread(image_file)
    if img is not None:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        img_yuv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YUV)
        img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
        equalized_img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
        return equalized_img
    return None

# Function to calculate channel average
def calculate_channel_average(img, channel):
    channel_dict = {"R": 0, "G": 1, "B": 2}
    channel_idx = channel_dict[channel]
    if img.ndim == 3 and img.shape[2] > channel_idx:
        channel_intensities = img[:, :, channel_idx].flatten()
        return np.mean(channel_intensities)
    else:
        return 0  # Return 0 for grayscale or unsupported images

# Load training images and labels
train_image_files, train_labels = load_images_and_labels(train_data_dir)

# Process images in batches to avoid memory overload
batch_size = 10
data = []

for i in range(0, len(train_image_files), batch_size):
    batch_files = train_image_files[i:i + batch_size]
    batch_labels = train_labels[i:i + batch_size]

    for image_file, label in zip(batch_files, batch_labels):
        img_np = equalize_image_cv2(image_file)
        if img_np is not None:
            red_avg = calculate_channel_average(img_np, "R")
            green_avg = calculate_channel_average(img_np, "G")
            blue_avg = calculate_channel_average(img_np, "B")
            data.append({'Label': label, 'Red': red_avg, 'Green': green_avg, 'Blue': blue_avg})

    # Save intermediate results to CSV to avoid memory overload
    temp_df = pd.DataFrame(data)
    temp_df.to_csv('/content/intermediate_equalized.csv', mode='a', header=not
                   os.path.exists('/content/intermediate_equalized.csv'), index=False)
    data.clear()  # Clear data to release memory

# Load all results from CSV
df = pd.read_csv('/content/intermediate_equalized.csv')

# Group by label and calculate mean intensities
mean_intensities = df.groupby("Label").mean().reset_index()

# Plotting function
def plot_channel_intensity_barplot(df, channel):
    title_dict = {"Red": "red", "Green": "green", "Blue": "blue"}
    palet_dict = {"Red": "Reds_d", "Green": "Greens_d", "Blue": "Blues_d"}
    plt.figure(figsize=(10, 3))

    values = df[channel].values
    pal = sns.color_palette(palet_dict[channel], len(values))
    rank = values.argsort().argsort()
    ax = sns.barplot(x=df["Label"], y=values, palette=np.array(pal[::-1])[rank])

    plt.ylabel("Intensity")
    plt.title(f"Average {title_dict[channel]} channel intensity for classes using Equalized Image",
              fontsize=15, fontweight='bold')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=55)
    plt.show()

# Plot the channel intensities
plot_channel_intensity_barplot(mean_intensities, "Red")
plot_channel_intensity_barplot(mean_intensities, "Green")
plot_channel_intensity_barplot(mean_intensities, "Blue")